# Tutorial on SupplyChain class in CLIMADA
https://climada-petals.readthedocs.io/en/latest/tutorial/climada_engine_SupplyChain.html

In [1]:
import numpy as np
import pandas as pd

from climada.util.api_client import Client
from climada_petals.engine import SupplyChain
from climada.entity import ImpfTropCyclone, ImpactFuncSet
from climada.engine.impact_calc import ImpactCalc
import datetime as dt

client = Client()

15:14:16 - boario - [INFO] - Unable to tell git branch as pygit2 was not found.
15:14:16 - boario - [INFO] - Loaded boario module. You can disable logging in console with `boario.disable_console_logging()`.


In [2]:
# Load in the exposure data
exp_usa = client.get_litpop('USA') 
# litpop provides the exposure of a population or asset
# here litpop is the assets exposure in the USA

In [3]:
# load in the hazard data
tc_usa = client.get_hazard('tropical_cyclone', properties={'country_iso3alpha':'USA', 'event_type': 'observed'})

target_year = 2017 # more target years take longer
events_in_target_year = np.array([
    tc_usa.event_name[i] for i in range(len(tc_usa.event_name)) if
    dt.datetime.fromordinal(tc_usa.date[i]).year == target_year
    ])

tc_usa_target_year = tc_usa.select(event_names = events_in_target_year)

In [4]:
# Define impact function
impf_tc = ImpfTropCyclone.from_emanuel_usa()
impf_set = ImpactFuncSet()
impf_set.append(impf_tc)
impf_set.check()

In [17]:
# Calculate direct impacts to the USA due to TC
imp_calc = ImpactCalc(exp_usa, impf_set, tc_usa_target_year)
direct_impact_usa = imp_calc.impact()
print(direct_impact_usa)

2025-04-22 16:40:54,204 - climada.util.coordinates - WARNING - Distance to closest centroid is greater than 100km for 19942 coordinates.


In [7]:
# can call or parse your own MRIOT
supchain = SupplyChain.from_mriot(mriot_type='WIOD16', mriot_year=2011)

898kKB [04:30, 3.31kKB/s]                           
/Users/cmor7802/miniforge3/envs/climada_env/lib/python3.11/site-packages/climada_petals/engine/supplychain.py:509: UserWarning: Found negatives values in total final demand, setting them to 0 and recomputing production vector
  warnings.warn("Found negatives values in total final demand, "


In [8]:
# regions
supchain.mriot.get_regions()

Index(['AUS', 'AUT', 'BEL', 'BGR', 'BRA', 'CAN', 'CHE', 'CHN', 'CYP', 'CZE',
       'DEU', 'DNK', 'ESP', 'EST', 'FIN', 'FRA', 'GBR', 'GRC', 'HRV', 'HUN',
       'IDN', 'IND', 'IRL', 'ITA', 'JPN', 'KOR', 'LTU', 'LUX', 'LVA', 'MEX',
       'MLT', 'NLD', 'NOR', 'POL', 'PRT', 'ROU', 'RUS', 'SVK', 'SVN', 'SWE',
       'TUR', 'TWN', 'USA', 'ROW'],
      dtype='object', name='region')

In [9]:
# sectors
supchain.mriot.get_sectors()

Index(['Crop and animal production, hunting and related service activities',
       'Forestry and logging', 'Fishing and aquaculture',
       'Mining and quarrying',
       'Manufacture of food products, beverages and tobacco products',
       'Manufacture of textiles, wearing apparel and leather products',
       'Manufacture of wood and of products of wood and cork, except furniture; manufacture of articles of straw and plaiting materials',
       'Manufacture of paper and paper products',
       'Printing and reproduction of recorded media',
       'Manufacture of coke and refined petroleum products ',
       'Manufacture of chemicals and chemical products ',
       'Manufacture of basic pharmaceutical products and pharmaceutical preparations',
       'Manufacture of rubber and plastic products',
       'Manufacture of other non-metallic mineral products',
       'Manufacture of basic metals',
       'Manufacture of fabricated metal products, except machinery and equipment',
       

In [10]:
# transaction matrix
supchain.mriot.Z

region                                                                                                                   AUS  \
sector                                                    Crop and animal production, hunting and related service activities   
region sector                                                                                                                  
AUS    Crop and animal production, hunting and related...                                       10954.209508                   
       Forestry and logging                                                                       113.905445                   
       Fishing and aquaculture                                                                     22.267974                   
       Mining and quarrying                                                                       232.437685                   
       Manufacture of food products, beverages and tob...                                        1662.225516                   
...                                                                                                      ...                   
ROW    Education                                                                                    1.305193                   
       Human health and social work activities                                                      0.535337                   
       Other service activities                                                                     1.856064                   
       Activities of households as employers; undiffer...                                           0.027816                   
       Activities of extraterritorial organizations an...                                           0.000000                   

region                                                                          \
sector                                                    Forestry and logging   
region sector                                                                    
AUS    Crop and animal production, hunting and related...           337.318807   
       Forestry and logging                                         127.867760   
       Fishing and aquaculture                                        0.025312   
       Mining and quarrying                                           2.323883   
       Manufacture of food products, beverages and tob...             4.046841   
...                                                                        ...   
ROW    Education                                                      0.022124   
       Human health and social work activities                        0.038688   
       Other service activities                                       0.058495   
       Activities of households as employers; undiffer...             0.000064   
       Activities of extraterritorial organizations an...             0.000000   

region                                                                             \
sector                                                    Fishing and aquaculture   
region sector                                                                       
AUS    Crop and animal production, hunting and related...              166.404066   
       Forestry and logging                                              0.588653   
       Fishing and aquaculture                                          21.871529   
       Mining and quarrying                                              8.353050   
       Manufacture of food products, beverages and tob...               78.987732   
...                                                                           ...   
ROW    Education                                                         0.093113   
       Human health and social work activities                           0.141195   
       Other service activities                                          0.149588   
       Activities of households as emp

In [11]:
# final demand
supchain.mriot.Y

region                                                                                            AUS  \
category                                                  Final consumption expenditure by households   
region sector                                                                                           
AUS    Crop and animal production, hunting and related...                                 7574.242904   
       Forestry and logging                                                                179.441312   
       Fishing and aquaculture                                                            1110.343558   
       Mining and quarrying                                                               3271.376748   
       Manufacture of food products, beverages and tob...                                32371.160467   
...                                                                                               ...   
ROW    Education                                                                            77.238254   
       Human health and social work activities                                              77.839965   
       Other service activities                                                             70.302003   
       Activities of households as employers; undiffer...                                   10.406364   
       Activities of extraterritorial organizations an...                                    0.000000   

region                                                                                                                                          \
category                                                  Final consumption expenditure by non-profit organisations serving households (NPISH)   
region sector                                                                                                                                    
AUS    Crop and animal production, hunting and related...                                                0.0                                     
       Forestry and logging                                                                              0.0                                     
       Fishing and aquaculture                                                                           0.0                                     
       Mining and quarrying                                                                              0.0                                     
       Manufacture of food products, beverages and tob...                                                0.0                                     
...                                                                                                      ...                                     
ROW    Education                                                                                         0.0                                     
       Human health and social work activities                                                           0.0                                     
       Other service activities                                                                          0.0                                     
       Activities of households as employers; undiffer...                                                0.0                                     
       Activities of extraterritorial organizations an...                                                0.0                                     

region                                                                                                 \
category                                                  Final consumption expenditure by government   
region sector                                                                                           
AUS    Crop and animal production, hunting and related...                                  405.543798   
       Forestry and logging                                     

In [12]:
# total production
supchain.mriot.x

indout
region sector                                                           
AUS    Crop and animal production, hunting and related...   69566.630223
       Forestry and logging                                  2719.293635
       Fishing and aquaculture                               3113.703184
       Mining and quarrying                                250207.439237
       Manufacture of food products, beverages and tob...   92033.592425
...                                                                  ...
ROW    Education                                           508147.713351
       Human health and social work activities             399362.733830
       Other service activities                            367396.468789
       Activities of households as employers; undiffer...   22003.013824
       Activities of extraterritorial organizations an...      33.142482

[2464 rows x 1 columns]

In [25]:
impacted_secs = supchain.mriot.get_sectors()[range(26,29)].tolist()
supchain.calc_shock_to_sectors(exp_usa, direct_impact_usa, impacted_secs)

In [26]:
supchain.mriot.unit

unit
region sector                                                   
AUS    Crop and animal production, hunting and related...  M.USD
       Forestry and logging                                M.USD
       Fishing and aquaculture                             M.USD
       Mining and quarrying                                M.USD
       Manufacture of food products, beverages and tob...  M.USD
...                                                          ...
ROW    Education                                           M.USD
       Human health and social work activities             M.USD
       Other service activities                            M.USD
       Activities of households as employers; undiffer...  M.USD
       Activities of extraterritorial organizations an...  M.USD

[2464 rows x 1 columns]

In [27]:
# exposure
print(
      exp_usa.gdf.value.sum() / supchain.conversion_factor(),
      "==",
      supchain.secs_exp.sum().sum(),
      "==",
      supchain.secs_exp.loc[:, ('USA', impacted_secs)].sum().sum(),
      )

# impact
print(
      direct_impact_usa.imp_mat.sum().sum() / supchain.conversion_factor(),
      "==",
      supchain.secs_imp.sum().sum(),
      "==",
      supchain.secs_imp.loc[:, ('USA', impacted_secs)].sum().sum(),
      )

65384554.30441263 == 65384554.304412626 == 65384554.304412626
141034.6512288862 == 141034.65122888572 == 141034.65122888572


In [28]:
supchain.secs_shock.loc[:, ('USA', impacted_secs)].head()

# unclear how we could set our own impact function? or shock - need to check the impactfunction

region            USA  \
sector   Construction   
event_id                
1446         0.000002   
1452         0.000074   
1454         0.002068   
1459         0.000012   

region                                                                            \
sector   Wholesale and retail trade and repair of motor vehicles and motorcycles   
event_id                                                                           
1446                                               0.000002                        
1452                                               0.000074                        
1454                                               0.002068                        
1459                                               0.000012                        

region                                                              
sector   Wholesale trade, except of motor vehicles and motorcycles  
event_id                                                            
1446                                               0.000002         
1452                                               0.000074         
1454                                               0.002068         
1459                                               0.000012

In [18]:
# let's try the first three events
for event_id in supchain.secs_shock.index[:3]:
    imp_event = direct_impact_usa.at_event[direct_impact_usa.event_id == event_id][0]
    print(imp_event / exp_usa.gdf.value.sum(), "==", supchain.secs_shock.loc[event_id, ('USA', impacted_secs)].values[0])

2.263715418906672e-06 == 2.263715418906672e-06
7.434784063419169e-05 == 7.434784063419169e-05
0.002068242123595854 == 0.002068242123595854


This practically means that the fraction of flow losses (on production/demand depending on the subsequent approach used) is assumed to be equal to the fraction of stock losses, since Impact and Exposure typically refer to stocks in CLIMADA. However, since depending on the sector one can reasonably expect production losses to be proportionally higher or lower than stock losses, a shock_factor argument can also be passed to calc_shock_to_sectors to define—for each sector—how much flow shocks should be higher/lower than stocks shocks (i.e., the mere Impact / Exposure ratio).

In [29]:
# You can create a custom shock but it requires extensive expertise
shock_factor = pd.DataFrame(np.repeat(1, supchain.mriot.x.shape[0]),
                            index=supchain.mriot.x.index,
                            columns=['shock'], dtype="float64")

# randomly generated for this tutorial
shock_facs_service_USA = np.array([
                                   0.38324804, 1.15930626, 0.38324804
                                   ])

shock_factor.loc[('USA', impacted_secs), :] = shock_facs_service_USA

# supply shock factors when calculating sectorial shocks
supchain.calc_shock_to_sectors(exp_usa, direct_impact_usa, impacted_secs, shock_factor.values.flatten())

In [30]:
supchain.secs_shock.loc[:, ('USA', impacted_secs)].head()

region             USA  \
sector    Construction   
event_id                 
1446      8.675645e-07   
1452      2.849366e-05   
1454      7.926497e-04   
1459      4.656036e-06   

region                                                                            \
sector   Wholesale and retail trade and repair of motor vehicles and motorcycles   
event_id                                                                           
1446                                               0.000003                        
1452                                               0.000086                        
1454                                               0.002398                        
1459                                               0.000014                        

region                                                              
sector   Wholesale trade, except of motor vehicles and motorcycles  
event_id                                                            
1446                                           8.675645e-07         
1452                                           2.849366e-05         
1454                                           7.926497e-04         
1459                                           4.656036e-06

In [31]:
supchain.calc_impacts(io_approach='leontief')

In [32]:
supchain.supchain_imp['leontief'].head()

region                                                                  AUS  \
sector   Crop and animal production, hunting and related service activities   
event_id                                                                      
1446                                               0.000029                   
1452                                               0.000939                   
1454                                               0.026120                   
1459                                               0.000153                   

region                                                                      \
sector   Forestry and logging Fishing and aquaculture Mining and quarrying   
event_id                                                                     
1446                 0.000004                0.000001             0.000859   
1452                 0.000115                0.000040             0.028199   
1454                 0.003205                0.001116             0.784446   
1459                 0.000019                0.000007             0.004608   

region                                                                 \
sector   Manufacture of food products, beverages and tobacco products   
event_id                                                                
1446                                               0.000023             
1452                                               0.000759             
1454                                               0.021118             
1459                                               0.000124             

region                                                                  \
sector   Manufacture of textiles, wearing apparel and leather products   
event_id                                                                 
1446                                               0.000011              
1452                                               0.000371              
1454                                               0.010310              
1459                                               0.000061              

region                                                                                                                                    \
sector   Manufacture of wood and of products of wood and cork, except furniture; manufacture of articles of straw and plaiting materials   
event_id                                                                                                                                   
1446                                               0.000007                                                                                
1452                                               0.000214                                                                                
1454                                               0.005945                                                                                
1459                                               0.000035                                                                                

region                                            \
sector   Manufacture of paper and paper products   
event_id                                           
1446                                    0.000021   
1452                                    0.000682   
1454                                    0.018976   
1459                                    0.000111   

region                                                \
sector   Printing and reproduction of recorded media   
event_id                                               
1446                                        0.000008   
1452                                        0.000246   
1454                                        0.006854   
1459                                        0.000040   

region                                                        ...  \
sector   Manufacture of coke and refined petroleum products 

In [33]:
supchain.supchain_imp['leontief'].loc[:,('CHE', slice(None))].max(0).sort_values(ascending=False)[:10]

region  sector                                                                                        
CHE     Insurance, reinsurance and pension funding, except compulsory social security                     0.584840
        Manufacture of computer, electronic and optical products                                          0.231866
        Wholesale trade, except of motor vehicles and motorcycles                                         0.181537
        Manufacture of basic pharmaceutical products and pharmaceutical preparations                      0.137811
        Manufacture of chemicals and chemical products                                                    0.124322
        Legal and accounting activities; activities of head offices; management consultancy activities    0.117616
        Manufacture of fabricated metal products, except machinery and equipment                          0.116696
        Manufacture of machinery and equipment n.e.c.                                       